In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
import logging
import gzip
import gensim 
import re
import spacy
import math
from bs4 import BeautifulSoup
from collections import Counter
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import string
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence, pad_sequence
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import nltk
from nltk.corpus import wordnet
from nltk.corpus import stopwords
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [4]:
print(torch.cuda.is_available())
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
device

True


'cuda:0'

# Preprocessing & Loading data

### Covid dataset

In [5]:
# Load dataset for clothing reviews
covid_train = pd.read_csv("/content/drive/MyDrive/data /Corona_NLP_test.csv", encoding="latin1")
covid_train = covid_train.dropna()
covid_test = pd.read_csv("/content/drive/MyDrive/data /Corona_NLP_test.csv")
covid_test = covid_test.dropna()

covid_trian_inds = list(range(0, covid_train.shape[0]))

# cat both datasets for preprocessing
frames = [covid_train, covid_test]
covid = pd.concat(frames)
print(covid_train.head())
print(covid_test.shape)
print(type(covid))

   UserName  ...           Sentiment
0         1  ...  Extremely Negative
1         2  ...            Positive
3         4  ...            Negative
4         5  ...             Neutral
5         6  ...             Neutral

[5 rows x 6 columns]
(2964, 6)
<class 'pandas.core.frame.DataFrame'>


In [6]:
covid.head()

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
0,1,44953,NYC,02-03-2020,TRENDING: New Yorkers encounter empty supermar...,Extremely Negative
1,2,44954,"Seattle, WA",02-03-2020,When I couldn't find hand sanitizer at Fred Me...,Positive
3,4,44956,Chicagoland,02-03-2020,#Panic buying hits #NewYork City as anxious sh...,Negative
4,5,44957,"Melbourne, Victoria",03-03-2020,#toiletpaper #dunnypaper #coronavirus #coronav...,Neutral
5,6,44958,Los Angeles,03-03-2020,Do you remember the last time you paid $2.99 a...,Neutral


In [7]:
# preprocess training and testing set

covid = covid[['OriginalTweet', 'Sentiment']]
covid.columns = ['tweet', 'sentiment']
sentiment_dict = {'negative': 0, 'extremely negative':0, 'neutral':1, 'positive':2, 'extremely positive':2}
error_line = []

def encode_sentiment(x):
    sent_encoded = sentiment_dict[x.lower()]
    return sent_encoded

covid['sentiment_encoded'] = covid['sentiment'].apply(lambda x: encode_sentiment(x))



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


In [8]:
covid.head()

,tweet,sentiment,sentiment_encoded
0,TRENDING: New Yorkers encounter empty supermar...,Extremely Negative,0
1,When I couldn't find hand sanitizer at Fred Me...,Positive,2
3,#Panic buying hits #NewYork City as anxious sh...,Negative,0
4,#toiletpaper #dunnypaper #coronavirus #coronav...,Neutral,1
5,Do you remember the last time you paid $2.99 a...,Neutral,1


In [9]:
print(len(covid.sentiment_encoded))
print(covid.shape[0])

5928
5928


#### Tokenize each sentence

In [10]:
# read in data
def clear_data(df):
    stopword = set(stopwords.words('english'))

    # tokenize and post process, remove unnecessary words appearing in tweets(like urls and usernames)
    tweet_words = []
    str_lis = []
    for t in df['tweet']:
        t = t.lower()
        tokens = tokenize(t)
        tmp = []
        for token in tokens:
            # if token in stopword:
            #     continue
            if not wordnet.synsets(token):
                continue
            # repalce usernames with @USER
            user = re.sub(r'@[\w\W]+', '@USER', token)
            # replace with URL
            url = re.sub(r'https[\w\W]+','URL',user)
            if url in stopword:
                continue
            tmp.append(url)
        tweet_words.append(tmp)
        st = ' '.join([str(item) for item in tmp])
        str_lis.append(st)
    df['tweet_words'] = tweet_words
    df['tweets'] = str_lis

    # remove duplicates
    df.drop_duplicates(subset=['tweets'], inplace=True)

    df.drop(columns='tweet_words', inplace=True)
    print(len(df))
    df.to_csv("covid.csv", index=False)
    return df

def tokenize(text):
    words = []
    # print(text.split())
    for token in text.split():
        # find wouldn't kind word
        words.extend(re.findall(r"\w+-\w+|https.+|\.+|\d+[\.,]\d+|[@#]\w+|[+-]\d+|(?:(?!n[’'])\w)+|\w?[’']\w+|[^\s\w]", token))
        # words.extend(re.findall(r"(?:(?!.')\w)+|\w?'\w+|[^\s\w]", token))
    return words

In [11]:
covid = clear_data(covid)

3086


In [12]:
covid.head()

,tweet,sentiment,sentiment_encoded,tweets
0,TRENDING: New Yorkers encounter empty supermar...,Extremely Negative,0,trending new encounter empty supermarket shelv...
1,When I couldn't find hand sanitizer at Fred Me...,Positive,2,find hand turned 2 pack check concerns driving...
3,#Panic buying hits #NewYork City as anxious sh...,Negative,0,buying hits city anxious shoppers stock food a...
4,#toiletpaper #dunnypaper #coronavirus #coronav...,Neutral,1,one week buying baby milk powder next buying t...
5,Do you remember the last time you paid $2.99 a...,Neutral,1,remember last time paid gallon regular gas pri...


#### Encode and truncate sentence

In [13]:
# tokenize the tweets into list
def encode_tweets(x):
    tweet_list = x.split()
    return tweet_list

covid['tweets_tokenized'] = covid['tweets'].apply(lambda x: encode_tweets(x))
covid.reset_index(drop=True, inplace=True)
# drop rows wth empty tokenized tweet
for i, row in covid.iterrows():
    if len(row.tweets_tokenized) < 2:
        covid.drop(i, inplace = True)

In [14]:
covid.head(30)

,tweet,sentiment,sentiment_encoded,tweets,tweets_tokenized
0,TRENDING: New Yorkers encounter empty supermar...,Extremely Negative,0,trending new encounter empty supermarket shelv...,"[trending, new, encounter, empty, supermarket,..."
1,When I couldn't find hand sanitizer at Fred Me...,Positive,2,find hand turned 2 pack check concerns driving...,"[find, hand, turned, 2, pack, check, concerns,..."
2,#Panic buying hits #NewYork City as anxious sh...,Negative,0,buying hits city anxious shoppers stock food a...,"[buying, hits, city, anxious, shoppers, stock,..."
3,#toiletpaper #dunnypaper #coronavirus #coronav...,Neutral,1,one week buying baby milk powder next buying t...,"[one, week, buying, baby, milk, powder, next, ..."
4,Do you remember the last time you paid $2.99 a...,Neutral,1,remember last time paid gallon regular gas pri...,"[remember, last, time, paid, gallon, regular, ..."
5,"@DrTedros ""We canÂt stop #COVID19 without pro...",Neutral,1,stop protecting prices surgical masks increase...,"[stop, protecting, prices, surgical, masks, in..."
6,Anyone been in a supermarket over the last few...,Extremely Positive,2,supermarket last days went normal shop last ni...,"[supermarket, last, days, went, normal, shop, ..."
7,Best quality couches at unbelievably low price...,Positive,2,best quality couches unbelievably low prices a...,"[best, quality, couches, unbelievably, low, pr..."
8,Beware of counterfeits trying to sell fake mas...,Extremely Negative,0,beware counterfeits trying sell fake masks che...,"[beware, counterfeits, trying, sell, fake, mas..."
9,Panic food buying in Germany due to #coronavir...,Extremely Negative,0,panic food buying germany due begun left behin...,"[panic, food, buying, germany, due, begun, lef..."


In [15]:
# sanity check for length of tweet
len_list = [len(x) for x in covid.tweets_tokenized]
max(len_list)

34

In [16]:
# get unique words in the corpus
all_words = []
for x in covid['tweets_tokenized']:
    all_words.extend(x)

word_set = list(set(all_words))
word_count = Counter(all_words)

# filter out words with low frequency
for word_list in covid.tweets_tokenized:
    new_list = []
    for word in word_list:
        if word_count[word] > 2:
            new_list.append(word)
    word_list = new_list

# update set of words after removing the ones with low frequency
new_word_list = []
for x in covid['tweets_tokenized']:
    new_word_list.extend(x)
word_set = list(set(new_word_list))

# map each unique words & unknown token in covid.encoded to an index
word2index = {}
word2index['<UNK>'] = 0
word2index['<PAD>'] = 1

for i, word in enumerate(word_set, 2):
    word2index[word] = i

# encode the original sequence
def encode(sent_list):
    result = []
    for x in sent_list:
        index = word2index[x]
        result.append(index)
    return result

covid['encoded_old'] = covid.tweets_tokenized.apply(lambda x: encode(x))


# get sequence average length
total_len = 0
for x in covid.encoded_old:
    total_len += len(x)

ave_len = math.floor(total_len/covid.shape[0])

# filter out long sequences --> encode all sequence to length = ave_len
# pad short sequence

def fix_tweet(x):
    size = min(len(x), ave_len)
    new_encoded = x[:size]
    len_before_pad = len(new_encoded)
    if size < ave_len:
        for j in range(0, ave_len - len(x)):
            new_encoded.append(0)
    return new_encoded

covid['encoded'] = covid.encoded_old.apply(lambda x: fix_tweet(x))
covid['tweet_length'] = covid.encoded_old.apply(lambda x: min(len(x), ave_len))

In [17]:
max(covid.tweet_length)

print(word2index['exchange'])
covid.head()

717


,tweet,sentiment,sentiment_encoded,tweets,tweets_tokenized,encoded_old,encoded,tweet_length
0,TRENDING: New Yorkers encounter empty supermar...,Extremely Negative,0,trending new encounter empty supermarket shelv...,"[trending, new, encounter, empty, supermarket,...","[181, 5875, 5518, 5788, 5077, 1123, 4754, 1008...","[181, 5875, 5518, 5788, 5077, 1123, 4754, 1008...",14
1,When I couldn't find hand sanitizer at Fred Me...,Positive,2,find hand turned 2 pack check concerns driving...,"[find, hand, turned, 2, pack, check, concerns,...","[1757, 770, 3485, 5274, 2063, 4411, 3990, 1020...","[1757, 770, 3485, 5274, 2063, 4411, 3990, 1020...",9
2,#Panic buying hits #NewYork City as anxious sh...,Negative,0,buying hits city anxious shoppers stock food a...,"[buying, hits, city, anxious, shoppers, stock,...","[554, 1668, 2549, 5835, 4526, 940, 2509, 4901,...","[554, 1668, 2549, 5835, 4526, 940, 2509, 4901,...",15
3,#toiletpaper #dunnypaper #coronavirus #coronav...,Neutral,1,one week buying baby milk powder next buying t...,"[one, week, buying, baby, milk, powder, next, ...","[2733, 6033, 554, 4360, 946, 6713, 2366, 554, ...","[2733, 6033, 554, 4360, 946, 6713, 2366, 554, ...",10
4,Do you remember the last time you paid $2.99 a...,Neutral,1,remember last time paid gallon regular gas pri...,"[remember, last, time, paid, gallon, regular, ...","[2197, 50, 1669, 3663, 1080, 167, 4447, 1144, ...","[2197, 50, 1669, 3663, 1080, 167, 4447, 1144, ...",13


In [18]:
# sanity check for encoding:
print(len(all_words))
print(ave_len)
lengths = [len(x) for x in covid.encoded]
print(max(lengths))
print(min(lengths))
print(set(covid.sentiment_encoded))
tweet_length = covid.tweet_length
covid.head()
print(len(word_set))

47853
15
15
15
{0, 1, 2}
6816


In [19]:
# Train test split from skearln
data_size = len(covid['encoded'])
assert data_size == len(covid['sentiment_encoded']) 
X, y = list(zip(list(covid['encoded']),(list(covid['tweet_length'])))), list(covid['sentiment_encoded'])
# X(data, length)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=43)

In [20]:
# Sanity check for length match
print(X_train[2])
print(y_train[2])

([6173, 5751, 2463, 2988, 6255, 2932, 2317, 1144, 0, 0, 0, 0, 0, 0, 0], 8)
2


## Dataset and Dataloader

In [21]:
class Dataset_word2vec(Dataset):
    def __init__(self, X, y):
        
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return  self.X[idx], self.y[idx]

In [22]:
trainSet_w2v = Dataset_word2vec(X_train, y_train)
testSet_w2v = Dataset_word2vec(X_test, y_test)

In [23]:
print(trainSet_w2v[2])

(([6173, 5751, 2463, 2988, 6255, 2932, 2317, 1144, 0, 0, 0, 0, 0, 0, 0], 8), 2)


In [24]:
def myCollate(batch):
    data = [item[0] for item in batch]
    label = [item[1] for item in batch]
    # sort sequence according to it's length in a batch
    data = data
    # data.sort(key=lambda x: x[1], reverse = True)

    # data (review: list, length: int)
    review = torch.tensor([x[0] for x in data], dtype = torch.long)
    label = torch.tensor(label, dtype = torch.long)
    seq_len = [x[1] for x in data]
    return review, label, seq_len

In [25]:
trainLoader_w2v = DataLoader(dataset = trainSet_w2v, batch_size = 16, collate_fn = myCollate, shuffle = True, drop_last = True)
testLoader_w2v = DataLoader(dataset = testSet_w2v, batch_size = 16, collate_fn = myCollate, shuffle = True, drop_last = True)

In [26]:
it = iter(trainLoader_w2v)
x, y, seq_len = it.next()

## Embedding

In [27]:
def load_glove_vectors(glove_file="/content/drive/MyDrive/data /glove.6B.300d.txt"):
    """Load the glove word vectors"""
    word_vectors = {}
    with open(glove_file) as f:
        for line in f:
            split = line.split()
            word_vectors[split[0]] = np.array([float(x) for x in split[1:]])
    return word_vectors



In [28]:
# create embedding matrix
def emb_matrix(model, all_words, word2index):
    matrix_size = len(word_set) + 2
    emb_matrix = np.zeros((matrix_size, 300))
    emb_matrix[0] = np.random.uniform(-0.25, 0.25, 300) # vector for UNK
    emb_matrix[1] = np.zeros(300) # vector for padding, has no weight
    invalid_len = 0
    # vector for every other word in the dictionary
    for i in range(2, len(all_words)+2):
        word = all_words[i-2]
        index = word2index[word]
        try:
            vector = model[word]
        except Exception as e:
            invalid_len += 1
            # word does not exist in the pretrained embedding
            vector = emb_matrix[0]
            #print(index)
        emb_matrix[index] = vector
    return emb_matrix, invalid_len

In [29]:
word2vec = load_glove_vectors()
data = list(word2vec.values())
an_array=np.array(data)


In [30]:
embedding, invalid_len= emb_matrix(word2vec, word_set, word2index)

In [31]:
def validation_metrics(model, valid_dl):
    print("current in vlaidaiton")
    model.eval()
    correct = 0
    total = 0
    sum_loss = 0.0
    sum_rmse = 0.0
    criterion = nn.CrossEntropyLoss()
    model = model.to(device)
    y_total = []
    y_pred_total = []
    for x, y, seq_len in valid_dl:
        x = x.long().to(device)
        y = y.long().to(device)
        y_hat = model(x, seq_len)
        loss = criterion(y_hat, y)
        
        # print(f"shape of y: {y.shape}")
        # print(f"shape of y_pred: {y_hat.shape}")
        # print(y_hat)
        pred = torch.argmax(y_hat, 1)
        y_total.extend(y.tolist())
        y_pred_total.extend(pred.tolist())
        
        # print(pred)
        correct += sum((pred == y))
        total += y.shape[0]
        sum_loss += loss.item()*y.shape[0]
        # print(len(y_total))
        # print(len(y_pred_total))

      # calculate precision, recall, and f1
    f1 = (f1_score(y_total, y_pred_total, average='weighted'))
    precision = (precision_score(y_total, y_pred_total, average='weighted'))
    recall =(recall_score(y_total, y_pred_total, average='weighted'))
    return sum_loss/total, correct/total, precision, recall, f1

In [32]:
def train_model(model, fname, epochs = 50, lr = 0.00003):
    parameters = filter(lambda p:p.requires_grad, model.parameters())
    optimizer = torch.optim.Adam(parameters, lr = lr)
    criterion = nn.CrossEntropyLoss()
    model = model.to(device)
    PATH = '/content/drive/MyDrive/data/' + fname
    epoch = 0
    best_val_acc = 0.0
    for i in range(epochs):
        epoch += 1
        print(f"At epoch {i}")
        batch_count = 0
        model.train()
        sum_loss = 0.0
        total = 0.0
        train_correct = 0.0
        for x, y, seq_len in tqdm(trainLoader_w2v):
            x = x.long().to(device)
            y = y.long().to(device)
            batch_count += 1
            optimizer.zero_grad()
            y_pred = model(x, seq_len)
            pred = torch.argmax(y_pred, 1)
            train_correct += sum((pred == y))
            loss = criterion(y_pred, y)
            loss.backward()
            optimizer.step()
            sum_loss += loss.item()*y.shape[0]
            total += y.shape[0]

        val_loss, val_acc, precision, recall, f1 = validation_metrics(model, testLoader_w2v)
        print("train loss %.3f, train accuracy %.3f, val loss %.3f, val accuracy %.3f, precision %.3f, recall %.3f, F1 %.3f" 
              % (sum_loss/total, train_correct/total, val_loss, val_acc, precision, recall, f1))
        if val_acc > best_val_acc and i>=2:
            best_val_acc = val_acc
            NEW_PATH = PATH+f'_{i}_{val_acc}'
            # torch.save(model.state_dict(), NEW_PATH)
            print(f"\t=> Best model saved at {i}th epoch with valication accuracy of {val_acc}")

In [33]:
train_examples = []
list1=list(word2vec.values())
for item in list1:
    in_tensor = torch.Tensor(item)
    train_examples.append(in_tensor)

In [34]:

weight = torch.FloatTensor(an_array)
word2vec_embedding = nn.Embedding.from_pretrained(weight)

class LSTM_Glove(torch.nn.Module):
    def __init__(self, emb_matrix, embedding_dim, hidden_dim):
        super().__init__()
        self.embedding_dim = embedding_dim
        self.embeddings = nn.Embedding(emb_matrix.shape[0], embedding_dim, padding_idx = 0)
        self.embeddings.weight.data.copy_(torch.from_numpy(emb_matrix))
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first = True)
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.linear = nn.Linear(hidden_dim, 3)
        self.dropout = nn.Dropout(0.5)

    def forward(self, x, seq_len):
        x = self.embeddings(x)
        x = self.dropout(x)
        output, (h_n, c_n) = self.lstm(x)
        output = output[:, -1, :]
        
        return self.linear(output)

In [35]:
model = LSTM_Glove(embedding, 300, 100)
train_model(model, fname='word2vec_lstm_covid', epochs=200)

At epoch 0


100%|██████████| 154/154 [00:00<00:00, 329.08it/s]


current in vlaidaiton
train loss 1.095, train accuracy 0.369, val loss 1.073, val accuracy 0.414, precision 0.444, recall 0.414, F1 0.340
At epoch 1


100%|██████████| 154/154 [00:00<00:00, 436.10it/s]
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


current in vlaidaiton
train loss 1.061, train accuracy 0.441, val loss 1.037, val accuracy 0.442, precision 0.371, recall 0.442, F1 0.403
At epoch 2


...

current in vlaidaiton
train loss 0.117, train accuracy 0.968, val loss 1.391, val accuracy 0.694, precision 0.698, recall 0.694, F1 0.688
At epoch 198


100%|██████████| 154/154 [00:00<00:00, 435.88it/s]


current in vlaidaiton
train loss 0.118, train accuracy 0.962, val loss 1.424, val accuracy 0.696, precision 0.705, recall 0.696, F1 0.689
At epoch 199


100%|██████████| 154/154 [00:00<00:00, 434.56it/s]

current in vlaidaiton
train loss 0.115, train accuracy 0.963, val loss 1.439, val accuracy 0.688, precision 0.696, recall 0.688, F1 0.680
